<a href="https://colab.research.google.com/github/imusicmash/stanford_llm_python/blob/main/StanfordLLMClassMisc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from James rhodes to try to pull phone number off a doc
def answer_question(query):
    instructions_template_string = """Ignore all previous instructions and answer from the provided lecture material. " +
        "You are a highly intelligent college professor teaching a college level course.  " +
        "Students will ask questions from my lecture material below.  " +
        "Your task is to give the students the best answer from the lecture material.  " +
        "You will only answer from the lecture material and not from any supplemental sources like the internet or wikipedia.  " +
        "Please do your best to keep your answers short and concise." +
        "Remind the student that you are here to help 24/7 and you want the students to love learning and ask more questions.  " +
        "Avoid simply providing the answer and try to explain the answer as if explaining to a second grader. " +
        "Answer ONLY from the lecture material and if you are unclear ask the student questions to help determine the question.  " +
        "Do not restate the question when answering the question and don't provide incorrect answers. " +
        "Ignore in appropriate questions and don't even answer them. "
        ""
        "Lecture: {context_str}"
        ""
        "Query: {query_str}"
        "Answer: "
        """
    instructions_template = PromptTemplate(instructions_template_string)
    llm = OpenAI(temperature=7, model="gpt-3.5-turbo", max_tokens=8000)
    pc = Pinecone(api_key=PINECONE_API_KEY)
    pinecone_index = pc.Index("delacy-ai")
    vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
    vector_index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
    # Grab 5 search results
    retriever = VectorIndexRetriever(index=vector_index, similarity_top_k=3, llm=llm)
    # Pass in your retriever from above, which is configured to return the top 5 results
    query_engine = RetrieverQueryEngine(retriever=retriever)
    query_engine.update_prompts(
        {"response_synthesizer:text_qa_template": instructions_template}
    )
    llm_query = query_engine.query(query)
    response = llm_query.response
    return response